In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    FFF_Train_Decile3 = pd.read_pickle("FFF_Train_Decile3.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile3.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile3, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 3')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 6s 2ms/step - loss: 0.4879 - acc: 0.7771 - val_loss: 0.4410 - val_acc: 0.8037
Epoch 36/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4818 - acc: 0.7751 - val_loss: 0.4408 - val_acc: 0.7958
Epoch 37/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4825 - acc: 0.7794 - val_loss: 0.4410 - val_acc: 0.7945

Epoch 00037: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 38/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4909 - acc: 0.7692 - val_loss: 0.4409 - val_acc: 0.7958
Epoch 39/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4844 - acc: 0.7836 - val_loss: 0.4409 - val_acc: 0.7958
Epoch 40/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4874 - acc: 0.7745 - val_loss: 0.4408 - val_acc: 0.7958
Epoch 41/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4832 - acc: 0.7777 - val_loss: 0.4408 - val_acc: 0.7958
Ep

Epoch 20/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4604 - acc: 0.7853 - val_loss: 0.4195 - val_acc: 0.8246
Epoch 21/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4609 - acc: 0.7899 - val_loss: 0.4194 - val_acc: 0.8272
Epoch 22/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4623 - acc: 0.7872 - val_loss: 0.4130 - val_acc: 0.8233
Epoch 23/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4597 - acc: 0.7882 - val_loss: 0.4188 - val_acc: 0.8168
Epoch 24/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4548 - acc: 0.7908 - val_loss: 0.4110 - val_acc: 0.8285
Epoch 25/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4592 - acc: 0.7872 - val_loss: 0.4157 - val_acc: 0.8168
Epoch 26/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4563 - acc: 0.7905 - val_loss: 0.4318 - val_acc: 0.7984
Epoch 27/75
3055/3055 [==============================] 

3055/3055 [==============================] - 6s 2ms/step - loss: 0.5481 - acc: 0.7300 - val_loss: 0.4940 - val_acc: 0.7880
Epoch 3/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.5165 - acc: 0.7607 - val_loss: 0.4651 - val_acc: 0.7919
Epoch 4/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.5055 - acc: 0.7682 - val_loss: 0.4571 - val_acc: 0.7932
Epoch 5/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.5017 - acc: 0.7624 - val_loss: 0.4547 - val_acc: 0.7906
Epoch 6/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4988 - acc: 0.7741 - val_loss: 0.4468 - val_acc: 0.7984
Epoch 7/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4952 - acc: 0.7637 - val_loss: 0.4482 - val_acc: 0.7814
Epoch 8/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4890 - acc: 0.7748 - val_loss: 0.4441 - val_acc: 0.7932
Epoch 9/75
3055/3055 [==============================] - 7s 2ms/step - los

Epoch 62/75
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4581 - acc: 0.7974 - val_loss: 0.4201 - val_acc: 0.8154
Epoch 63/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4479 - acc: 0.8003 - val_loss: 0.4201 - val_acc: 0.8154
Epoch 64/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4525 - acc: 0.7997 - val_loss: 0.4205 - val_acc: 0.8181
Epoch 65/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4532 - acc: 0.7944 - val_loss: 0.4201 - val_acc: 0.8168
Epoch 66/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4491 - acc: 0.7984 - val_loss: 0.4201 - val_acc: 0.8154
Epoch 67/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4517 - acc: 0.7967 - val_loss: 0.4198 - val_acc: 0.8181
Epoch 68/75
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4529 - acc: 0.7961 - val_loss: 0.4199 - val_acc: 0.8168
Epoch 69/75
3055/3055 [==============================] 

...

3055/3055 [==============================] - 6s 2ms/step - loss: 0.4715 - acc: 0.7813 - val_loss: 0.4280 - val_acc: 0.8181
Epoch 79/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4711 - acc: 0.7797 - val_loss: 0.4281 - val_acc: 0.8168
Epoch 80/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4373 - acc: 0.8069 - val_loss: 0.4015 - val_acc: 0.8246

Epoch 00041: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 42/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4398 - acc: 0.8046 - val_loss: 0.4015 - val_acc: 0.8246
Epoch 43/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4307 - acc: 0.8020 - val_loss: 0.4017 - val_acc: 0.8246
Epoch 44/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4319 - acc: 0.8029 - val_loss: 0.4017 - val_acc: 0.8246
Epoch 45/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4308 - acc: 0.8023 - val_loss: 0.4019 - val_acc: 0.8233


3055/3055 [==============================] - 6s 2ms/step - loss: 0.4708 - acc: 0.7892 - val_loss: 0.4344 - val_acc: 0.8154

Epoch 00048: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 49/125
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4700 - acc: 0.7876 - val_loss: 0.4337 - val_acc: 0.8154
Epoch 50/125
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4682 - acc: 0.7944 - val_loss: 0.4334 - val_acc: 0.8168
Epoch 51/125
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4767 - acc: 0.7843 - val_loss: 0.4331 - val_acc: 0.8181
Epoch 52/125
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4689 - acc: 0.7954 - val_loss: 0.4328 - val_acc: 0.8168
Epoch 53/125
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4711 - acc: 0.7921 - val_loss: 0.4329 - val_acc: 0.8181
Epoch 54/125
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4697 - acc: 0.7931 - val_loss: 0.4324 - val_acc: 0.8

Epoch 21/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4405 - acc: 0.8003 - val_loss: 0.4037 - val_acc: 0.8298
Epoch 22/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4456 - acc: 0.8033 - val_loss: 0.4058 - val_acc: 0.8259
Epoch 23/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4402 - acc: 0.7984 - val_loss: 0.4022 - val_acc: 0.8298
Epoch 24/100
3055/3055 [==============================] - 3s 874us/step - loss: 0.4434 - acc: 0.8046 - val_loss: 0.4071 - val_acc: 0.8233
Epoch 25/100
3055/3055 [==============================] - 3s 916us/step - loss: 0.4339 - acc: 0.8003 - val_loss: 0.4042 - val_acc: 0.8246
Epoch 26/100
3055/3055 [==============================] - 2s 811us/step - loss: 0.4369 - acc: 0.8046 - val_loss: 0.4055 - val_acc: 0.8233
Epoch 27/100
3055/3055 [==============================] - 3s 829us/step - loss: 0.4391 - acc: 0.7977 - val_loss: 0.4085 - val_acc: 0.8272
Epoch 28/100
3055/3055 [================

Epoch 79/100
3055/3055 [==============================] - 2s 784us/step - loss: 0.4320 - acc: 0.8049 - val_loss: 0.4050 - val_acc: 0.8194
Epoch 80/100
3055/3055 [==============================] - 2s 783us/step - loss: 0.4367 - acc: 0.7997 - val_loss: 0.4049 - val_acc: 0.8194
Epoch 81/100
3055/3055 [==============================] - 2s 777us/step - loss: 0.4310 - acc: 0.8007 - val_loss: 0.4048 - val_acc: 0.8168
Epoch 82/100
3055/3055 [==============================] - 2s 786us/step - loss: 0.4396 - acc: 0.8026 - val_loss: 0.4046 - val_acc: 0.8181
Epoch 83/100
3055/3055 [==============================] - 2s 784us/step - loss: 0.4352 - acc: 0.8052 - val_loss: 0.4048 - val_acc: 0.8194
Epoch 84/100
3055/3055 [==============================] - 2s 784us/step - loss: 0.4321 - acc: 0.8039 - val_loss: 0.4046 - val_acc: 0.8194
Epoch 85/100
3055/3055 [==============================] - 2s 786us/step - loss: 0.4353 - acc: 0.8065 - val_loss: 0.4047 - val_acc: 0.8194
Epoch 86/100
3055/3055 [==========

3055/3055 [==============================] - 2s 769us/step - loss: 0.4658 - acc: 0.7823 - val_loss: 0.4329 - val_acc: 0.8154
Epoch 38/50
3055/3055 [==============================] - 3s 828us/step - loss: 0.4664 - acc: 0.7817 - val_loss: 0.4325 - val_acc: 0.8154
Epoch 39/50
3055/3055 [==============================] - 3s 857us/step - loss: 0.4662 - acc: 0.7745 - val_loss: 0.4327 - val_acc: 0.8154
Epoch 40/50
3055/3055 [==============================] - 2s 763us/step - loss: 0.4691 - acc: 0.7804 - val_loss: 0.4325 - val_acc: 0.8154
Epoch 41/50
3055/3055 [==============================] - 2s 768us/step - loss: 0.4685 - acc: 0.7781 - val_loss: 0.4323 - val_acc: 0.8168
Epoch 42/50
3055/3055 [==============================] - 2s 756us/step - loss: 0.4686 - acc: 0.7840 - val_loss: 0.4325 - val_acc: 0.8154
Epoch 43/50
3055/3055 [==============================] - 2s 765us/step - loss: 0.4681 - acc: 0.7761 - val_loss: 0.4325 - val_acc: 0.8154
Epoch 44/50
3055/3055 [==============================

In [3]:
0.93484

0.93484